In [1]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.03727865219116211  seconds 


In [6]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [7]:


def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object



In [8]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [9]:
def read_xml(li):
    
    
    all_objects=[]
    for i in range(len(li)):
    # print("i will run ", i+1," times")
        name_of_file=clean_file_names(li[i])
    # print(name_of_file)
        data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

        post_text='  '
        tree = etree.parse(li[i], parser=parser)
        root = tree.getroot()
        for element in root.iter():
            if element.tag == 'post':
                element_text = element.text
                post_text = post_text + element_text
        mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
        mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



        data_object['posts']=mystring   
        all_objects.append(data_object)

    return all_objects

In [10]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [11]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(posts):
    filtered_post=[]
    
    word_tokens = word_tokenize(posts) 
    filtered_sentence = []   
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    filtered_post.append(filtered_sentence)
    return filtered_post

In [12]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered    

In [13]:
for i in range(len(data)):
    data[i]['cleaned_stop_words']=fun_remove_stop_words(data[i]['posts'])

In [14]:
def remove_dashes(data):
    for i in range(len(data)):
        data[i]['cleaned_stop_words']=data[i]['cleaned_stop_words'].replace('--', ' ')
        

In [15]:
remove_dashes(data)

In [16]:
df=pd.DataFrame(data)

In [17]:
X=df[['age','cleaned_stop_words']]

In [18]:
dummy_variable_1 = pd.get_dummies(df["category"])
X=pd.concat([X,dummy_variable_1], axis=1)

In [19]:
del dummy_variable_1

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [21]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(X['cleaned_stop_words']).toarray()

In [ ]:
# vectorizer = CountVectorizer()
# vectorizer.fit(X['cleaned_stop_words'])
# posts = vectorizer.transform(X['cleaned_stop_words'])


In [109]:
posts

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 3, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 4, 0, 2]])

In [27]:
# transformed_posts = np.array([row.toarray() for row in posts])

In [32]:
# transformed_posts.shape=(100,35680)

In [23]:
transformed_posts=pd.DataFrame(posts)

In [24]:
X=pd.concat([X,transformed_posts],axis=1)

In [25]:
del transformed_posts

In [26]:
X=X.drop(['cleaned_stop_words'], axis=1)

In [27]:
X

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,15,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,24,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,15,0,0,0,0,0,0,0,0,0,...,0,0,9,17,4,25,0,14,2,3
14596,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14597,16,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14598,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [28]:
X['age']=X['age'].astype(int)

In [29]:
from scipy import stats

In [30]:
t1=time.time()

X['age'] = stats.zscore(X['age'])
    
t2=time.time()
print("time ::: ", t2-t1 , " seconds to find z score")

time :::  0.0010628700256347656  seconds to find z score


In [31]:
X.head()

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,-1.002358,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,0.252330,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,0.126861,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0


In [32]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [33]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [34]:
labels = mlb.fit_transform(labels)

In [35]:
y=labels

In [36]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, random_state=42)

### NN

In [37]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [38]:


input_dim = trainX.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [39]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                20840     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 14)                14350     
Total params: 50,760
Trainable params: 48,712


In [40]:
trainX, testX, trainY, testY
classifier_nn = model.fit(trainX,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(testX, testY),
                    batch_size=10)






Train on 11680 samples, validate on 2920 samples
Epoch 1/5





11680/11680 [==============================] - 8s 689us/step - loss: 0.3855 - acc: 0.8571 - val_loss: 0.4932 - val_acc: 0.8571
Epoch 2/5
11680/11680 [==============================] - 5s 447us/step - loss: 0.3801 - acc: 0.8575 - val_loss: 1.1099 - val_acc: 0.7988
Epoch 3/5
11680/11680 [==============================] - 5s 452us/step - loss: 0.3799 - acc: 0.8578 - val_loss: 0.6840 - val_acc: 0.8591
Epoch 4/5
11680/11680 [==============================] - 5s 446us/step - loss: 0.3795 - acc: 0.8582 - val_loss: 0.4775 - val_acc: 0.8575
Epoch 5/5
11680/11680 [==============================] - 5s 448us/step - loss: 0.3788 - acc: 0.8590 - val_loss: 0.5960 - val_acc: 0.8591


In [41]:
testX.shape

(2920, 1041)

In [42]:
testX[2].shape

(2920,)

In [43]:
model.predict(testX[1])

ValueError: Error when checking input: expected dense_1_input to have shape (1041,) but got array with shape (1,)

In [44]:
prediction=model.predict(testX)

In [49]:
for i in prediction:
    print(i)
   

[0.00496591 0.00496268 0.02532187 0.0062688  0.03662114 0.0056411
 0.02888441 0.01853754 0.01380137 0.05930358 0.00275195 0.02338019
 0.00243881 0.7671206 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497029 0.00496351 0.02530694 0.0062646  0.03662302 0.00563841
 0.02893148 0.01852819 0.01379915 0.05935797 0.00274912 0.02338706
 0.00244352 0.76703674]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00510124 0.00515221 0.02633027 0.00639093 0.03760651 0.00587705
 0.02974794 0.01921457 0.01429323 0.06003829 0.00288471 0.02395079
 0.00261918 0.7607931 ]
[0.00497057 0.00496393 0.02530917 0.00626488 0.036625

 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00412016 0.00405853 0.02197101 0.00546925 0.03226794 0.00465643
 0.02217842 0.01600943 0.01183667 0.05222197 0.00236619 0.02073903
 0.00148156 0.8006235 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[3.1902075e-03 3.0833448e-03 1.7850570e-02 4.5993035e-03 2.6668601e-02
 3.5509155e-03 1.5150629e-02 1.2799076e-02 9.4804522e-03 4.3524671e-02
 1.9076376e-03 1.7187817e-02 7.4993976e-04 8.4025693e-01]
[0.00496543 0.00495824 0.02528789 0.00626018 0.03659872 0.00563268
 0.02889156 0.01851381 0.01378793 0.05931913 0.00274689 0.02337233
 0.00243709 0.7672281 ]
[0.00497005 0.00496321 0.02530562 0.00626437 0.03662155 0.00563806
 0.02892969 0.01852725 0.01379845 0.05935629 0.00274895 0.02338618
 0.0024432

 0.00244335 0.7670431 ]
[0.00393204 0.00382758 0.02168445 0.00565368 0.02934525 0.00433913
 0.01929619 0.01443054 0.01102513 0.04814245 0.00229456 0.01834549
 0.00128398 0.8163996 ]
[0.00555494 0.00590337 0.03049443 0.00710262 0.04117354 0.00687773
 0.03183722 0.02190099 0.01632196 0.06172052 0.00349781 0.02574597
 0.00331677 0.73855203]
[0.00510485 0.00522055 0.02781778 0.00664015 0.03686173 0.00588317
 0.02821316 0.01881793 0.01420214 0.05762603 0.00297035 0.02269598
 0.00259848 0.76534766]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.025

 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.0253059  0.00626446 0.03662196 0.00563816
 0.02893062 0.01852748 0.01379864 0.05935721 0.00274898 0.02338641
 0.00244335 0.7670434 ]
[0.00496543 0.00495827 0.02528804 0.0062604  0.03659771 0.00563257
 0.02889035 0.01851302 0.01378757 0.05931787 0.00274684 0.023371
 0.00243704 0.7672339 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530589 0.00626446 0.03662198 0.00563816
 0.02893063 0.01852748 0.01379864 0.05935721 0.00274898 0.02338644
 0.00244335 0.7670433 ]
[0.00497015 0.00496331 0.02530

In [76]:
testX.shape

(2920, 1041)

In [78]:
testX.iloc[1].shape

(1041,)

In [79]:
testX.iloc[1].dtypes

dtype('float64')

In [81]:
testX.iloc[1]

age            -1.002358
Accounting      0.000000
Advertising     0.000000
Agriculture     0.000000
Architecture    0.000000
                  ...   
995             6.000000
996             1.000000
997             1.000000
998             0.000000
999             0.000000
Name: 9354, Length: 1041, dtype: float64

In [75]:
sample_test=testX.iloc[1]
# sample_test.shape=(1,1041)
model.predict(sample_test)

ValueError: Error when checking input: expected dense_1_input to have shape (1041,) but got array with shape (1,)

In [70]:
sample_test.transpose()

age             0.377799
Accounting      0.000000
Advertising     0.000000
Agriculture     0.000000
Architecture    0.000000
                  ...   
995             0.000000
996             0.000000
997             0.000000
998             0.000000
999             0.000000
Name: 169, Length: 1041, dtype: float64

### End of NN

## ML

In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
neigh = KNeighborsClassifier(n_neighbors=4)

In [52]:
neigh.fit(trainX,trainY)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [55]:
neigh.predict_proba(testX)

[array([[1.  , 0.  ],
        [0.75, 0.25],
        [1.  , 0.  ],
        ...,
        [1.  , 0.  ],
        [1.  , 0.  ],
        [1.  , 0.  ]]),
 array([[0.75, 0.25],
        [1.  , 0.  ],
        [1.  , 0.  ],
        ...,
        [1.  , 0.  ],
        [0.75, 0.25],
        [1.  , 0.  ]]),
 array([[1.  , 0.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        ...,
        [1.  , 0.  ],
        [1.  , 0.  ],
        [0.75, 0.25]]),
 array([[0.75, 0.25],
        [1.  , 0.  ],
        [1.  , 0.  ],
        ...,
        [0.5 , 0.5 ],
        [0.75, 0.25],
        [0.5 , 0.5 ]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1.  , 0.  ],
        [1.  , 0.  ],
        [0.75, 0.25],
        ...,
        [1.  , 0.  ],
        [0.75, 0.25],
        [1.  , 0.  ]]),
 array([[0.75, 0.25],
        [1.  , 0.  ],
        [0.75, 0.25],
        ...,
        [0.75, 0.25],
        [0.75, 0.25],
        [1.  , 0.  

### Word2 Vector implementation

In [ ]:
words_of_posts=[]
for j in range(len(data)):
    posts=data[j]['posts']
    
    words=[]
    for i in range(len(posts)):
        words.append(word_tokenize(posts[i]))
    words_of_posts.append(words)

In [ ]:
remove_stop_words(data[1]['posts'])

In [ ]:
if len(data)==len(words_of_posts):
    for i in range(len(data)):
        data[i]['tokenized']=words_of_posts[i]

In [ ]:
df=pd.DataFrame(data)

In [ ]:
X=df['tokenized'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
y=data['horoscope']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

In [ ]:
y_transform=le.transform(y)

In [ ]:
from keras.utils import np_utils

In [ ]:
dummy_y = np_utils.to_categorical(y_transform)

In [ ]:
#Split

In [ ]:
# model

In [ ]:
# compile

In [ ]:
# fit

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [ ]:
freq_matrix=_create_frequency_matrix(data[1]['posts'])

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [ ]:
tf_matrix=_create_tf_matrix(freq_matrix)

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
doc_per_word=_create_documents_per_words(freq_matrix)

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
_create_idf_matrix(freq_matrix,doc_per_word,total_documents)